# Robinet de Ransom 2D homogène

In [ ]:
from trustutils import run

run.introduction('A. Drouillet (TMA), J. Francescatto (CEA)')
run.description("Cas tests du robinet de Ramson avec découpage des phases GAZ et LIQUIDE.")


## Description du cas test
Le probléme de Ransom 1D a été présenté par V.H. Ransom dans. Ce cas test "raide" permet de tester la stabilité du schéma, sa précision et l'interaction avec des termes sources simples. Il représente l'écoulement 1D d'une colonne d'eau verticale soumise à la force de gravité. L'eau accélérant sous l'effet de la gravité, le diamétre du jet doit diminue pour assurer la conservation du débit. Le cas test consiste en une collone de 12 m de hauteur et 1 m de largeur contenant de l'eau et de l'air, traités comme des fluides incompressibles. Ces fluides obeissent à la loi d'état Stiffened Gas avec pour :
* l'air : Pinf=0 et gamma = 1.4
* l'eau : Pinf=6 10^8 et gamma = 4.4


Dans cette fiche de validation, il est regardé les résultats obtenus lorsque chaque phase (GAZ et LIQUIDE) est séparée en plusieurs sous-phases (4 pour GAZ et 8 pour LIQUIDE). Cela permet de valider la programmation informatique de la définition de sous phases dans le jeu de données. Cette base informatique servira pour la mise en place de d'une méthode multi groupes de type MUSIG.

In [ ]:
run.TRUST_parameters()

In [ ]:
from trustutils import run, plot
import os

run.reset()



run.defaultSuite_ = run.TRUSTSuite(runPrepare=False)


origin, build_dir = os.getcwd(), run.BUILD_DIRECTORY
os.chdir(build_dir)
exec(open("mainParameter.py").read())



os.chdir(origin)

for case in names:
    if(TCs[case]["nombre procs"] > 1):
        run.addCase("%s" % (case) ,"%s.data" %case,nbProcs=TCs[case]["nombre procs"])
    else :
        run.addCase("%s" % (case) ,"%s.data" %case)
    
run.printCases()
run.runCases()

In [ ]:
run.tablePerf()

In [ ]:
display(tab)

In [ ]:
# Approximate exact solution 
import math
import numpy as np

def exactAlphaGas(x,t):
    g = 10
    L = 12
    U0 = 10
    alpha0 = 0.2
    if(x<U0*t+0.5*g*t*t):
        return 1-((1-alpha0)*U0)/(math.sqrt(U0*U0+2*g*x))
    else:
        return alpha0

def exactVitesseLiquid(x,t):
    g = 10
    L = 12
    U0 = 10
    alpha0 = 0.2
    if(x<U0*t+0.5*g*t*t):
        return -math.sqrt(U0*U0+2*g*x)
    else:
        return -(U0+g*t)
    
def writeSolution(fichierName, Dt, function, N):
    L = 12
    xRange = np.linspace(0,L,N)
    fichier = open(fichierName, "w")
    for i in xRange :
        x=L-i
        ligne = str(x) + " " + str(function(i, Dt)) + " \n" 
        fichier.write(ligne)
    fichier.close()

In [ ]:
import time
from trustutils import plot
import matplotlib.pyplot as plt

#time step
timeStep = [0.3, 0.6, 0.8]
Ncases = len(timeStep)
for Dt in timeStep : 
    fichierNameAlpha = run.BUILD_DIRECTORY + "/AlphaExact" + str(Dt) + ".txt"
    fichierNameVitesse = run.BUILD_DIRECTORY+"/VitesseExact" + str(Dt) + ".txt"
    writeSolution(fichierNameAlpha, Dt, exactAlphaGas, 50)
    writeSolution(fichierNameVitesse, Dt, exactVitesseLiquid, 50)
    
a = plot.Graph("Resultats", nX=Ncases, nY=2)
    
line = 0
for t in timeStep:
    a.addPlot([line,0],"Taux de vide à t = %s" %t)
    
    a.addSegment("ransom_StiffenedGas/ransom_StiffenedGas_SONDE_ALPHA_GAZ_HELIUM.son",lw=3,time=t,label="ransom nbPhases %s" %TCs["ransom_StiffenedGas"]["nb_Phases"])
    a.addSegment("ransom_StiffenedGasMUSIG/ransom_StiffenedGasMUSIG_SONDE_ALPHA_GAZ_HELIUM_0.son",lw=3,time=t,label="ransom nbPhases %s" %TCs["ransom_StiffenedGasMUSIG"]["nb_Phases"])
    a.addSegment("ransom_StiffenedGasMUSIG/ransom_StiffenedGasMUSIG_SONDE_ALPHA_GAZ_HELIUM_1.son",lw=3,time=t,label="ransom nbPhases %s" %TCs["ransom_StiffenedGasMUSIG"]["nb_Phases"])
    a.addSegment("ransom_StiffenedGasMUSIG/ransom_StiffenedGasMUSIG_SONDE_ALPHA_GAZ_HELIUM_2.son",lw=3,time=t,label="ransom nbPhases %s" %TCs["ransom_StiffenedGasMUSIG"]["nb_Phases"])
    a.addSegment("ransom_StiffenedGasMUSIG/ransom_StiffenedGasMUSIG_SONDE_ALPHA_GAZ_HELIUM_3.son",lw=3,time=t,label="ransom nbPhases %s" %TCs["ransom_StiffenedGasMUSIG"]["nb_Phases"])

    data = plot.loadText("AlphaExact" + str(t) + ".txt")
    a.add(data[0],data[1],label="Analytique",lw=3,marker='o',color='k')
    a.label("y en m","Taux de vide")
    
    a.addPlot([line,1],"Vitesse du fluide à t = %s" %t)
    
    a.addSegment("ransom_StiffenedGas/ransom_StiffenedGas_SONDE_VITESSE_LIQUIDE_SODIUM.son",lw=3,compo=1,time=t,label="ransom nbPhases %s" %TCs["ransom_StiffenedGas"]["nb_Phases"])
    a.addSegment("ransom_StiffenedGasMUSIG/ransom_StiffenedGasMUSIG_SONDE_VITESSE_LIQUIDE_SODIUM_0.son",lw=3,compo=1,time=t,label="ransom nbPhases %s" %TCs["ransom_StiffenedGasMUSIG"]["nb_Phases"])
    a.addSegment("ransom_StiffenedGasMUSIG/ransom_StiffenedGasMUSIG_SONDE_VITESSE_LIQUIDE_SODIUM_1.son",lw=3,compo=1,time=t,label="ransom nbPhases %s" %TCs["ransom_StiffenedGasMUSIG"]["nb_Phases"])
    a.addSegment("ransom_StiffenedGasMUSIG/ransom_StiffenedGasMUSIG_SONDE_VITESSE_LIQUIDE_SODIUM_2.son",lw=3,compo=1,time=t,label="ransom nbPhases %s" %TCs["ransom_StiffenedGasMUSIG"]["nb_Phases"])
    a.addSegment("ransom_StiffenedGasMUSIG/ransom_StiffenedGasMUSIG_SONDE_VITESSE_LIQUIDE_SODIUM_3.son",lw=3,compo=1,time=t,label="ransom nbPhases %s" %TCs["ransom_StiffenedGasMUSIG"]["nb_Phases"])
    a.addSegment("ransom_StiffenedGasMUSIG/ransom_StiffenedGasMUSIG_SONDE_VITESSE_LIQUIDE_SODIUM_4.son",lw=3,compo=1,time=t,label="ransom nbPhases %s" %TCs["ransom_StiffenedGasMUSIG"]["nb_Phases"])
    a.addSegment("ransom_StiffenedGasMUSIG/ransom_StiffenedGasMUSIG_SONDE_VITESSE_LIQUIDE_SODIUM_5.son",lw=3,compo=1,time=t,label="ransom nbPhases %s" %TCs["ransom_StiffenedGasMUSIG"]["nb_Phases"])
    a.addSegment("ransom_StiffenedGasMUSIG/ransom_StiffenedGasMUSIG_SONDE_VITESSE_LIQUIDE_SODIUM_6.son",lw=3,compo=1,time=t,label="ransom nbPhases %s" %TCs["ransom_StiffenedGasMUSIG"]["nb_Phases"])
    a.addSegment("ransom_StiffenedGasMUSIG/ransom_StiffenedGasMUSIG_SONDE_VITESSE_LIQUIDE_SODIUM_7.son",lw=3,compo=1,time=t,label="ransom nbPhases %s" %TCs["ransom_StiffenedGasMUSIG"]["nb_Phases"])

    
    data = plot.loadText("VitesseExact" + str(t) + ".txt")
    a.add(data[0],data[1],label="Analytique",lw=3,marker='o',color='k')
    a.label("y en m","Vitesse du fluide m/s")
    line += 1    
    
plt.savefig(build_dir + "/ransom.png", bbox_inches='tight', facecolor='white')